# Weather Data API

In [1]:
import pandas as pd
import time
import requests
import json


In [12]:
latlong = pd.read_csv('./latlongpunjab.csv')

In [13]:
latlong['Latitude decimal'] = round(latlong['Latitude decimal'],3)
latlong['Longitude decimal'] = round(latlong['Longitude decimal'],3)
latlong['latlong'] = latlong['Latitude decimal'].map(str) + ','+ latlong['Longitude decimal'].map(str)
latlong.head()

,Location,Latitude,Longitude,Latitude decimal,Longitude decimal,Econ Data Locations,latlong
0,Amritsar,31° 37' N,74° 55' E,31.617,74.917,Amritsar,"31.617,74.917"
1,Barnala,NaN,NaN,30.382,75.547,Barnala,"30.382,75.547"
2,Bathinda,30° 11' N,75° 00' E,30.183,75.000,Bathinda,"30.183,75.0"
3,Faridkot,NaN,NaN,30.677,74.758,Faridkot,"30.677,74.758"
4,Fatehgarh Sahib,NaN,NaN,30.681,76.410,Fatehgarh Sahib,"30.681,76.41"


In [3]:
jala = latlong = pd.read_csv('./Jalandhar.csv')

In [4]:
jala

,Location,Latitude,Longitude,Latitude decimal,Longitude decimal,Econ Data Locations
0,Jalandhar,NaN,NaN,31.2569,75.4432,Jalandhar


In [7]:
# Weather Company Cleaned Historical Python starter code - adjusted for Punjab India pull
# Used a trial key with limit 15K API calls
# https://docs.google.com/document/d/1OdMo3Z7qre0cvJftPRtx0Xi8MEfvA0Z77tw1GsZ8gYg/edit

import time
import requests, json,csv,math
import string

###############################################################################

def APIcall(lat,lon,row): # function for pulling data
    base1 = 'http://cleanedobservations.wsi.com/CleanedObs.svc/premium?version=2&lat='
    base2 = lat+"&long="+lon+"&startDate="+startd+"&endDate="+endd
    base3 = '&interval=daily&units=metric&format=json&userKey='
#     api_key = 'insert key here'
    url = base1 + base2 + base3 + api_key # make API URL

    newdata = []
    for x in row:
        newdata.append(x)
  
    data = requests.get(url).json() #API Call
    lat2 = data['head']['site']['latitude']
    lon2 = data['head']['site']['longitude']


    for x in data['weatherData']['dailyAverages']['averages']:
        row = []
        row = row + newdata
        row.append(lat)
        row.append(lon)
        row.append(x["year"])
        row.append(int(x["month"]))
        row.append(int(x["day"]))

        #data harvesting and assignment
        hr1_precip = x['precipitationPreviousHourCentimeters']
        runoff = x['surfaceWaterRunOffMillimeters']
        soil_10 = x['zeroToTenLiquidSoilMoisturePercent']
        soil_40 = x['tenToFortyLiquidSoilMoisturePercent']
        soil_100 = x['fortyToOneHundredLiquidSoilMoisturePercent']

        #apppending harvested data to a row to write to csv

        row.append(hr1_precip)
        row.append(runoff)
        row.append(soil_10)
        row.append(soil_40)
        row.append(soil_100)
        print(newdata,x["day"],x["month"])

        #write row to csv
        writer.writerow(row)

##############################################################################

def col_convert(col): #converts columns letters to numbers
    col = str(col)
    #maxes out with a 2 character column...doesn't allow 3 or more
    if len(col) >= 3:
        print("ERROR: Maximum of 2 letters in column ID")
        quit()
    #does the column letter to number conversion
    num = 0
    for t in col:
        if t in string.ascii_letters:
            num = num * 26 + (ord(t.upper()) - ord('A')) + 1
    num = num - 1
    return num

###############################################################################
ih = "./latlongfile.csv"
# startd = input('Enter your start date (MM/DD/YYYY): ')
startd = "01/01/2008"
startd = str(startd)
try:
    startmonth = int(startd[:2])
    startday = int(startd[3:5])
    startyr = int(startd[6:])
except:
    print("the date was likely entered incorrectly")
    quit()

if len(str(startyr)) != 4 or len(startd) != 10 or startmonth > 12 or startday > 31:
    print("your date entry was not correct")
    quit()

endd = "10/30/2019"
# endd = input('Enter your end date (MM/DD/YYYY): ')
# endd = str(endd)
try:
    endmonth = int(endd[:2])
    endday = int(endd[3:5])
    endyr = int(endd[6:])
except:
    print("the date was likely entered incorrectly")
    quit()

if len(str(endyr)) != 4 or len(endd) != 10 or endmonth > 12 or endday > 31:
    print("your date entry was not correct")
    quit()

col = input('Enter the column letter for the latitude in the input file: ')
collat = col_convert(col)
col = input('Enter the column letter for the longitude in the input file: ')
collon = col_convert(col)

#checks if the startyr and endyr is a leap year
if startyr % 4 == 0:
    slpyr = True
else:
    slpyr = False
if endyr % 4 == 0:
    elpyr = True
else:
    elpyr = False

#checks to see if the Feb 29 date is valid if no leap yr
if (slpyr or elpyr) is False and (startmonth or endmonth) is 2 and (startday or endday) > 28:
    print("the date does not exist")
    quit()

if endmonth in [1,3,5,7,8,10] and endday == 31:
    endday = 1
    endmonth = endmonth + 1
elif endmonth in [4,6,9,11] and endday == 30:
    endday = 1
    endmonth = endmonth + 1
elif endmonth == 2 and endday == 28 and elpyr == False:
    endmonth = 3
    endday = 1
elif endmonth == 2 and endday == 28 and elpyr == True:
    endmonth = 2
    endday = 29
elif endmonth is 2 and endday == 29 and elpyr == True:
    endmonth = 3
    endday = 1
elif endmonth == 12 and endday == 31:
    endmonth = 1
    endday = 1
    endyr = endyr + 1
else:
    endday = endday + 1

startd = str(startmonth) + '/' + str(startday) + '/' + str(startyr)
endd = str(endmonth) + '/' + str(endday) + '/' + str(endyr)

input_file = csv.reader(open(ih,'r'))
df='./weatherdata/weather20082019.csv'
output = open(df, 'w',newline='')  # open file
colnames = ['Lat','Lon','Year','Month','Day','Past 1hour Precipitation(in)','Runoff(mm)','Soil Moisture% (0-10)',
            'Soil Moisture% (10-40)','Soil Moisture% (40-100)']
writer = csv.writer(output)

i=0
headcol = []

for irow in input_file:

    if i == 0:
        writer.writerow(irow + colnames)
        i = i + 1
        continue

    row = []
    for x in irow:
        row.append(x)

    lat = irow[collat]
    lon = irow[collon]

    APIcall(lat,lon,row)

input('\nThe script is finished running.  Press any key to continue...')

Enter the column letter for the latitude in the input file: B
Enter the column letter for the longitude in the input file: C
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 1 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 2 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 3 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 4 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 5 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 6 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 7 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 8 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 9 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 10 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 11 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 12 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 13 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 14 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 15 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 16 1
['Jalandhar', '31.25

['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 31 1
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 1 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 2 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 3 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 4 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 5 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 6 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 7 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 8 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 9 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 10 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 11 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 12 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 13 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 14 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 15 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 16 2
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 17 2
['Jalandhar', '31.2569', '75.4432', '

['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 4 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 5 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 6 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 7 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 8 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 9 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 10 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 11 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 12 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 13 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 14 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 15 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 16 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 17 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 18 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 19 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 20 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 21 4
['Jalandhar', '31.2569', '75.4432'

['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 24 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 25 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 26 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 27 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 28 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 29 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 30 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 31 10
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 1 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 2 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 3 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 4 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 5 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 6 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 7 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 8 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 9 11
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 10 11
['Jalandhar', '31.2

['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 27 3
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 28 3
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 29 3
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 30 3
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 31 3
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 1 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 2 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 3 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 4 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 5 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 6 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 7 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 8 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 9 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 10 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 11 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 12 4
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 13 4
['Jalandhar', '31.2569', '75.4432', '

['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 14 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 15 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 16 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 17 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 18 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 19 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 20 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 21 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 22 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 23 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 24 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 25 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 26 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 27 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 28 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 29 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 30 9
['Jalandhar', '31.2569', '75.4432', 'Jalandhar'] 1 10
['Jalandhar', '31.2569', '75

''

In [39]:
weather_test = pd.read_csv("./weatherdata/weather20082019.csv")
weather_test.shape

(86985, 12)

In [40]:
weather_test.head()

,lat,long,Lat,Lon,Year,Month,Day,Past 1hour Precipitation(in),Runoff(mm),Soil Moisture% (0-10),Soil Moisture% (10-40),Soil Moisture% (40-100)
0,31.617,74.917,31.617,74.917,2009,1,1,"{'min': 0.0, 'max': 0.0, 'avg': 0.0, 'sum': 0.0}","{'min': 0.0, 'max': 0.00155, 'avg': 0.00037, '...","{'min': 15.5, 'max': 16.1, 'avg': 16.0}","{'min': 18.8, 'max': 19.0, 'avg': 18.8}","{'min': 19.4, 'max': 19.6, 'avg': 19.4}"
1,31.617,74.917,31.617,74.917,2009,1,2,"{'min': 0.0, 'max': 0.0, 'avg': 0.0, 'sum': 0.0}","{'min': 0.0, 'max': 0.00257, 'avg': 0.00048, '...","{'min': 15.9, 'max': 16.8, 'avg': 16.5}","{'min': 18.7, 'max': 18.8, 'avg': 18.7}","{'min': 19.3, 'max': 19.4, 'avg': 19.3}"
2,31.617,74.917,31.617,74.917,2009,1,3,"{'min': 0.0, 'max': 0.082, 'avg': 0.00254, 'su...","{'min': 0.0, 'max': 0.0116, 'avg': 0.0016, 'su...","{'min': 16.4, 'max': 20.0, 'avg': 17.5}","{'min': 18.7, 'max': 18.7, 'avg': 18.7}","{'min': 19.3, 'max': 19.3, 'avg': 19.3}"
3,31.617,74.917,31.617,74.917,2009,1,4,"{'min': 0.0, 'max': 0.053, 'avg': 0.00254, 'su...","{'min': 0.0, 'max': 0.00544, 'avg': 0.00098, '...","{'min': 20.1, 'max': 24.5, 'avg': 23.4}","{'min': 18.7, 'max': 19.0, 'avg': 18.8}","{'min': 19.3, 'max': 19.3, 'avg': 19.3}"
4,31.617,74.917,31.617,74.917,2009,1,5,"{'min': 0.0, 'max': 0.0, 'avg': 0.0, 'sum': 0.0}","{'min': 0.0, 'max': 0.00232, 'avg': 0.00044, '...","{'min': 22.9, 'max': 24.4, 'avg': 23.2}","{'min': 18.8, 'max': 19.0, 'avg': 18.9}","{'min': 19.2, 'max': 19.3, 'avg': 19.2}"
